In [18]:
from numpy import load
import pickle
import scipy
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_learning_curves
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import RandomOverSampler


# Look into partial fitting. 
# Dataset is not evenly distributed with most being 1.
# Dummy Classifier has 50 % accuracy score... (Only give instances the most frequent label)



# steps_countvec = scipy.sparse.load_npz('recipe_text_features_countvec/train_steps_countvectorizer.pkl')

# Doc2Vect is a technique to transfer words into numerical representation. 
# https://www.shibumi-ai.com/post/a-gentle-introduction-to-doc2vec
d2v_ingr = pd.read_csv("../data/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_ingr_doc2vec100.csv", header=None)
d2v_name = pd.read_csv("../data/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_name_doc2vec100.csv", header=None)
d2v_steps = pd.read_csv("../data/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_steps_doc2vec100.csv", header=None)



# Extract class_labels from training set 
# quick = 1
# medium = 2
# slow = 3
data_train = pd.read_csv('../data/COMP30027_2021_Project2_datasets/recipe_train.csv')

# Obtain the labels
train_label = data_train.iloc[:,-1]




# Feature selection with f_classif (ANOVA F-value)
# ANOVA f-value shows how well a feature discriminate between classes
# The more discrimination, the better that feature is in predicting the class label.
# d2v_name_new = SelectKBest(k=90).fit_transform(d2v_name, train_label)
# d2v_ingr_new = SelectKBest(k=90).fit_transform(d2v_ingr, train_label)
# d2v_steps_new = SelectKBest(k=90).fit_transform(d2v_steps, train_label)
d2v_name_new = pd.DataFrame(d2v_name)
d2v_ingr_new = pd.DataFrame(d2v_ingr)
d2v_steps_new = pd.DataFrame(d2v_steps)
# print(d2v_ingr_new)
# print(d2v_steps_new)
# print(d2v_name_new.shape)


# Create a new dataframe of data, but this time, name, steps and ingr has been engineered to have doc2vec features.
# 100 doc2vec features were given, but we selected 20 best features using ANOVA f-value.
f_data = d2v_name_new.join(data_train.iloc[:,1:3], on=None, how='left', lsuffix='_left', rsuffix='_right')
f_data = f_data.join(d2v_steps_new, on=None, how='left', lsuffix='_left', rsuffix='_right')
f_data = f_data.join(d2v_ingr_new, on=None, how='left', lsuffix='_left', rsuffix='_right')

# # Standardise the data so that the mean is 0
# scaler = StandardScaler()
# f_data = scaler.fit_transform(f_data)

# normalise all values to be between 0 and 1
minmax_scaler = MinMaxScaler()
f_data = minmax_scaler.fit_transform(f_data)


X_new = SelectKBest(chi2, k=20).fit_transform(f_data, train_label)

# Splitting the provided training into its own train/test
X_train, X_test, y_train, y_test = train_test_split(X_new, train_label, test_size=0.2, stratify=train_label, random_state=42)

smote = SMOTE(random_state=42)
smX, smY = smote.fit_resample(X_train, y_train)
# # Oversampling, because the distribution of classes in training data is highly skewed towards quick and medium.
# oversample = RandomOverSampler(sampling_strategy='minority')

# # First random oversampling, bring count of class 3 to be equal to the highest class count.
# X_oversampled, y_oversampled = oversample.fit_resample(X_train, y_train)

# # Second random oversampling, bring the count of class 2 to be equal to the other 2 classes
# X_oversampled, y_oversampled = oversample.fit_resample(X_oversampled, y_oversampled)



# X_oversampled = pd.DataFrame(X_oversampled)
# y_oversampled = pd.DataFrame(y_oversampled)
# X_oversampled.to_csv('X_oversampled', index = False)
# y_oversampled.to_csv('y_oversampled', index = False)


# X = pd.DataFrame(X_oversampled)
# y = pd.DataFrame(y_oversampled)

# full = X.join(y, on=None, how='left', lsuffix='_left', rsuffix='_right')
smX

array([[0.44111683, 0.0617284 , 0.02564103, ..., 0.51126455, 0.37345331,
        0.3947126 ],
       [0.3907844 , 0.0617284 , 0.12820513, ..., 0.4447808 , 0.46763176,
        0.52414391],
       [0.43168394, 0.04938272, 0.25641026, ..., 0.33210021, 0.47459023,
        0.34165161],
       ...,
       [0.56981364, 0.05296908, 0.2597055 , ..., 0.50625011, 0.50360741,
        0.33506021],
       [0.5161363 , 0.21205649, 0.36047591, ..., 0.48089585, 0.44863147,
        0.26215384],
       [0.5430582 , 0.14418386, 0.19179803, ..., 0.40271453, 0.65661782,
        0.49141788]])

In [ ]:
# Hyper parameter tuning

In [19]:
# # Plot learning curve


# LinearSVC = svm.LinearSVC(dual=False, multi_class='ovr', random_state=0)
# SVC_clf = make_pipeline(MinMaxScaler(), LinearSVC)
SVC_clf = svm.LinearSVC(dual=False, multi_class='ovr', random_state=0)
# plot_learning_curves(smX, smY, X_test, y_test, SVC_clf, scoring= 'accuracy')
# plt.show()

In [20]:
# Implementing LinearSVM from sklearn library
# Similar to SVC with parameter kernel=’linear’, 
# scale better to large numbers of samples.
# supports both dense and sparse input
# multiclass support is handled according to a one-vs-the-rest scheme.
# The more features the better
from sklearn.pipeline import make_pipeline
# LinearSVC = svm.LinearSVC(dual=False, multi_class='ovr', random_state=0)

SVC_clf.fit(smX, smY)

y_test_predict = SVC_clf.predict(X_test)
y_train_predict = SVC_clf.predict(smX)
# np.save('y_predict_name_sparse.npy', y_test_predict)
print("test", accuracy_score(y_test, y_test_predict))
print("train", accuracy_score(smY, y_train_predict))
# print(LinearSVC_clf.score(X_test, y_test))




# 0 score for precision, recall and f1 on class label 3...
print(classification_report(y_test, y_test_predict))
cm = confusion_matrix(y_test, y_test_predict)
cm

test 0.566375
train 0.5926817723446729
              precision    recall  f1-score   support

         1.0       0.66      0.64      0.65      3541
         2.0       0.73      0.50      0.59      4049
         3.0       0.14      0.60      0.22       410

    accuracy                           0.57      8000
   macro avg       0.51      0.58      0.49      8000
weighted avg       0.67      0.57      0.60      8000



array([[2264,  685,  592],
       [1064, 2022,  963],
       [  93,   72,  245]], dtype=int64)

In [ ]:
# Cross Validation To evaluate model
# Shuffle Split is a random split method, not guranteed unique split but likely on large data.
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
scores = cross_val_score(LinearSVC_clf, X_oversampled, y_oversampled, cv=cv)
scores

In [ ]:
from sklearn.linear_model import SGDClassifier

clf_SGD = SGDClassifier(max_iter=1000, tol=1e-3, n_jobs=6, random_state=0)
plot_learning_curves(X_train, y_train, X_test, y_test, clf_SGD, scoring= 'accuracy')
plt.show()